### QSE for the N2H2 molecule
Using Qiskit and Pyscf

In this notebook QSE algorithm (Quantum Subspace Expansion) is implemented and used to find both the first excited state and the ground state of the $N_{2}$$H_{2}$ molecule.

The results of QSE are then compared to the classical solver of Pyscf, a lightweight, modular platform for quantum chemistry and solid-state calculations.

In contrast to the previous notebook for $H_{2}$ we will find specifically what we call the gap, but to find this gap we will find the energy of ground and first excited states.

### Outline
<ol>
    <li>Role of N2H2 in fertilizers</li>
    <li>Required installs</li>
    <li>Algorithm steps</li>
    <li>Required imports</li>
    <li>Building the hamiltonian</li>
    <li>Building the ansatz and running VQE</li>
    <li>Building the excitation operators and subspace matrices</li>
    <li>Running the soluition</li>
    <li>Classical solution using Pyscf</li>
    <li>Conclusion</li>
    <li>References</li>
    <li>Versions</li>
</ol>

### Role of N2H2 in fertilizers
<hr>

Today, roughly 80% of the annually produced ammonia $NH_{3}$ is used for fertilizer production. $N_{2}$ is first transformed into diazene ($N_{2}H_{2}$) before being transformed into ammonia as shown in the following two equations.

$\text{N}_2 + 2\text{H}^+ + 2\text{e}^- \rightarrow \text{N}_2\text{H}_2$

$\text{N}_2\text{H}_2 + 4\text{H}^+ + 4\text{e}^- \rightarrow 2\text{NH}_3$

We can design better catalyst for generating Ammonia from diazene and this is very important as 1-2% of annual energy is used in fertilizers production.

Diazene (N₂H₂) is a fragile and reactive intermediate in nitrogen reduction pathways. Without the right environment, it can decompose, follow unproductive routes, or simply fail to convert into ammonia (NH₃). This is where the photocatalyst becomes essential.

A well-designed photocatalyst absorbs light and generates excited electrons that can be transferred to diazene, pushing it toward further reduction. But for this transfer to be efficient, the photocatalyst’s electronic structure — particularly its conduction band edge — must align with diazene’s excited-state energy levels.

Knowing the energy gap between diazene’s ground and first excited state allows researchers to:

1. Tune the photocatalyst’s band structure to match diazene’s reactivity window

2. Stabilize diazene long enough for it to undergo productive transformation

3. Avoid energy losses due to mismatched excitation or unwanted side reactions

4. Select materials (like TiO₂, g-C₃N₄, or metal-organic frameworks) that interact optimally with diazene’s electronic states

In short, the photocatalyst acts as both an energy mediator and a reaction guide, and understanding diazene’s excited-state behavior is what allows it to do that job well. This synergy is what makes the transformation from N₂H₂ to NH₃ faster, cleaner, and more efficient — a key step toward sustainable nitrogen fixation.

### Required installs
<hr>

The required installation steps are shown in the github Readme file also. If installed there then no need to reinstall here.

In [40]:
#!pip install qiskit['all']==1.4.4

In [41]:
#!pip install qiskit-nature-pyscf

In [42]:
#!pip install qiskit-aer

### Algorithm steps
<hr>

QSE is an algorithm for finding excited states of a given Hamiltonian $H$. It resembles the configuration interaction method in quantum chemistry.

1. ***Excitation Operators:***
Determine a set of excitation operators $E_1, \dots, E_M$ and a reference (approximate) ground state $|\psi_{GS}\rangle$. Single excitations of electrons, $c^\dagger_j c_l$ ($j, l = 0, 1, \dots$), are one of the common choices of the excitation operators. For notational convenience, we add $E_0 = I$ (identity operator) to the set of excitation operators. Note that $E_0 |\psi_{GS}\rangle = |\psi_{GS}\rangle$.

2. ***Ground state energy calculation:***
Prepare the approximate ground state $|\psi_{GS}\rangle$, obtained by the VQE algorithm (or other methods), on quantum computers.

3. ***Build the $S$ and $h$ matrices:***
Measure quantities $h_{ij} = \langle \psi_{GS} | E_i^\dagger H E_j | \psi_{GS} \rangle$ and $S_{ij} = \langle \psi_{GS} | E_i^\dagger E_j | \psi_{GS} \rangle$ on quantum computers ($i, j = 0, \dots, M$).

4. ***Diagonalization:***
Diagonalize the Hamiltonian within the subspace spanned by $E_0 |\psi_{GS}\rangle, \dots, E_M |\psi_{GS}\rangle$. Namely, solve a generalized eigenvalue problem within the subspace:  
$hC = SC E'$,  where $C$ is the coefficient vector for (approximate) eigenvectors and $E'$ is a diagonal matrix whose diagonal elements are (approximate) eigenvalues of $H$.

It is noted that the QSE also plays a role in mitigating noise errors inevitable in NISQ devices (see reference below).


### Required imports
<hr>

In [43]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import Statevector
from scipy.linalg import eigh
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit.circuit.library import TwoLocal
import numpy as np

from pyscf import gto, scf
from pyscf import fci

The following cell is to get rid of warnings like deprecated qiskit versions, for example.

In [44]:
import warnings
warnings.filterwarnings('ignore')

### Building the hamiltonian
<hr>

The $N_{2}H_{2}$ hamiltonian is built by specifying the four constituting atoms with their coordinates in 3D.

The electorns that will get excited are that on the outer levels so we will freeze the first twol orbitals. This will make the calculated energy different from that in the classical without freezing but as mentioned, the goal is to calculate the energy gap specifically and not the exact energy of each level.

This will be shown later.

In [45]:
# Step 1: Define N2H2 molecule and apply orbital freezing
driver = PySCFDriver(atom="N 0.0 0.0 0.0; N 0.0 0.0 1.2; H 0.0 0.9 -0.4; H 0.0 0.9 1.6", basis="sto3g")
problem = driver.run()

# Freeze 2 core orbitals, keep 4 spatial orbitals (8 spin orbitals)
transformer = ActiveSpaceTransformer(num_electrons=4, num_spatial_orbitals=4)
problem = transformer.transform(problem)

# Step 2: Map to qubit Hamiltonian
mapper = JordanWignerMapper()
qubit_op = mapper.map(problem.hamiltonian.second_q_op())

In [46]:
qubit_op

SparsePauliOp(['IIIIIIII', 'IIIIIIIZ', 'IIIIIIZI', 'IIIIIZII', 'IIIIZIII', 'IIIZIIII', 'IIZIIIII', 'IZIIIIII', 'ZIIIIIII', 'IIIIIIZZ', 'IIIIIZIZ', 'IIIIZIIZ', 'IIIZIIIZ', 'IIZIIIIZ', 'IZIIIIIZ', 'ZIIIIIIZ', 'IIIIYYYY', 'IIIIXXYY', 'IIIIYXXY', 'IIIIXYYX', 'IIIIYYXX', 'IIIIXXXX', 'IIYYIIYY', 'IIXXIIYY', 'IIYYIIXX', 'IIXXIIXX', 'YYIIIIYY', 'XXIIIIYY', 'YYIIIIXX', 'XXIIIIXX', 'IYZYIYZY', 'IXZXIYZY', 'IYZYIXZX', 'IXZXIXZX', 'YZYIIYZY', 'XZXIIYZY', 'YZYIIXZX', 'XZXIIXZX', 'YZZYYZZY', 'XZZXYZZY', 'YZZYXZZX', 'XZZXXZZX', 'IYYIYZZY', 'IXXIYZZY', 'IYYIXZZX', 'IXXIXZZX', 'IIIIIZZI', 'IIIIZIZI', 'IIIZIIZI', 'IIZIIIZI', 'IZIIIIZI', 'ZIIIIIZI', 'YZZYIYYI', 'XZZXIYYI', 'YZZYIXXI', 'XZZXIXXI', 'IYYIIYYI', 'IXXIIYYI', 'IYYIIXXI', 'IXXIIXXI', 'IYZYYZYI', 'IXZXYZYI', 'IYZYXZXI', 'IXZXXZXI', 'YZYIYZYI', 'XZXIYZYI', 'YZYIXZXI', 'XZXIXZXI', 'IIIIZZII', 'IIIZIZII', 'IIZIIZII', 'IZIIIZII', 'ZIIIIZII', 'IIYYYYII', 'IIXXYYII', 'IIYYXXII', 'IIXXXXII', 'YYIIYYII', 'XXIIYYII', 'YYIIXXII', 'XXIIXXII', 'IIIZZIII', '

### Building the ansatz and running VQE
<hr>

The eigenstate and eigenvalue got from the ground state will be used in the next steps for finding the excited states.

In [47]:
# Step 3: Build hardware-efficient ansatz and run VQE
num_qubits = qubit_op.num_qubits
ansatz = TwoLocal(num_qubits, rotation_blocks='ry', entanglement_blocks='cz', reps=2, insert_barriers=True)

vqe_solver = VQE(StatevectorEstimator(), ansatz, COBYLA(maxiter=100))
vqe_result = vqe_solver.compute_minimum_eigenvalue(qubit_op)
params = vqe_result.optimal_parameters

### Building the excitation operators and subspace matrices
<hr>

Two important points are worth mentioning here.

1. Only single excited states operators will be built and used. Other operators like double excitation and spin operators for example can be built. But the simplest case of single excitations will be used for simplicity.

2. We have used the simulator to perform VQE in the previous cell, since we will not be using noisy simulators or running on real hardware, we can implement expectation values using tensors and multiplication in the following cell for finding the exited states. This is the same method used by the statevector simulator.

In [48]:
# Step 4: Generate filtered single excitation operators (occupied → virtual only)
def generate_filtered_excitations_manual(problem):
    # Manually define occupied and virtual spin orbital indices
    occ_indices = [0, 1, 2, 3]       # First 4 spin orbitals
    virt_indices = [4, 5, 6, 7]      # Last 4 spin orbitals

    ops = []
    for i in occ_indices:
        for j in virt_indices:
            label = f"+_{j} -_{i}"
            ops.append(FermionicOp({label: 1.0}, num_spin_orbitals=problem.num_spin_orbitals))
    return ops

excitation_ops = generate_filtered_excitations_manual(problem)
qubit_excitations = [mapper.map(op) for op in excitation_ops]

# Step 5: Prepare ground state vector
state = Statevector(ansatz.assign_parameters(params).decompose().to_instruction())

In [49]:
# Step 6: Build subspace matrices
def build_matrices(ops, H, psi):
    N = len(ops)
    S = np.zeros((N, N))
    H_mat = np.zeros((N, N))
    for i in range(N):
        psi_i = ops[i].to_matrix() @ psi.data
        for j in range(N):
            psi_j = ops[j].to_matrix() @ psi.data
            S[i, j] = np.vdot(psi_i, psi_j).real
            H_mat[i, j] = np.vdot(psi_i, H.to_matrix() @ psi_j).real
    return H_mat, S

H_sub, S_sub = build_matrices(qubit_excitations, qubit_op, state)

### Running the solution
<hr>

In [50]:
# Step 7: Solve generalized eigenvalue problem
epsilon = 1e-6
S_reg = S_sub + epsilon * np.eye(S_sub.shape[0])
eigvals, eigvecs = eigh(H_sub, S_reg)
print("Excited state energies from QSE:", eigvals)

Excited state energies from QSE: [-4.55833513 -4.45540248 -4.39173454 -4.21572903 -4.20566044 -4.20323486
 -4.15022979 -4.07849765 -4.03818874 -3.97128756 -3.91112332 -3.88220336
 -3.86021147 -3.82874223 -3.67469598 -3.5896942 ]


### Classical solution using pyscf:
<hr>

In [51]:
# Define N2H2 molecule
mol = gto.M(
    atom="N 0.0 0.0 0.0; N 0.0 0.0 1.2; H 0.0 0.9 -0.4; H 0.0 0.9 1.6",
    basis="sto-3g",
    spin=0,
    charge=0
)

# Run Hartree-Fock
mf = scf.RHF(mol)
mf.kernel()

# Instantiate the FCI solver
fci_solver = fci.FCI(mf)

# Run Full CI (FCI)
fci_energies, _ = fci_solver.kernel(nroots=5)

print("🔬 FCI Energies (Hartree):")
for i, e in enumerate(fci_energies):
    print(f"  State {i}: {e:.6f}")

converged SCF energy = -108.524268245802
🔬 FCI Energies (Hartree):
  State 0: -108.664894
  State 1: -108.547690
  State 2: -108.501255
  State 3: -108.427769
  State 4: -108.409634


In [52]:
# FCI excitation gaps
fci_gaps = [e - fci_energies[0] for e in fci_energies]
# QSE excitation gaps
qse_gaps = [e - eigvals[0] for e in eigvals]

In [53]:
fci_gaps

[np.float64(0.0),
 np.float64(0.11720466133641594),
 np.float64(0.1636396281062389),
 np.float64(0.23712520819680094),
 np.float64(0.2552604406120338)]

In [54]:
qse_gaps

[np.float64(0.0),
 np.float64(0.10293264764496879),
 np.float64(0.1666005913696349),
 np.float64(0.3426060988634543),
 np.float64(0.3526746868350399),
 np.float64(0.3551002652031725),
 np.float64(0.4081053375025254),
 np.float64(0.479837475284544),
 np.float64(0.5201463872505023),
 np.float64(0.5870475672246735),
 np.float64(0.6472118042469299),
 np.float64(0.6761317720629263),
 np.float64(0.6981236613763184),
 np.float64(0.7295929014494074),
 np.float64(0.8836391509405233),
 np.float64(0.9686409303105776)]

### Conclusion:
<hr>

In this notebook we have shown the following:
<ol>
    <li>How to build the hamiltonian operator of a molecule.</li>
    <li>How to calculate the ground state energy using VQE.</li>
    <li>How to calculate the first excited state using QSE.</li>
</ol>

\begin{array}{|c|c|c|}
\hline
\textbf{Algorithm} & \textbf{Gap ground vs 1st excited} & \textbf{Gap ground vs 2nd excited}\\
\hline
\text{QSE} & {0.102} & {0.166} \\
\text{Pyscf} & {0.117} & {0.163} \\
\hline
\end{array}

As said before, for designing this photocatalyst the gap between the ground and first excited states of diazene is the most important and not the exact energy.

By freezing the first two orbitals time was hugely reduced as without this reduction the QSE algorithm took 6 hours and did not even finish while with this reduction it takes few seconds only.

It is totally reasonable to get different energies from the classical solution compared to QSE since in QSE the first two orbitals were freezed but the gap energies should be near.

This huge decrease in energy between the two solvers is because the orbitals near to the nucleus which were freezed hold the highest energy.

In [58]:
print("Error qse_gap vs pyscf_gap ground vs 1st excited = ", (0.117 - 0.102)/0.117 * 100, "%")

Error qse_gap vs pyscf_gap ground vs 1st excited =  12.82051282051283 %


This error is good knowing that we are not using the best ansatz and optimizer or starting possible parameters. With other modifications this error can decrease a lot. Also, this error changes from run to run as VQE is not deterministic.

### References
<hr>

<ol>
    <li>Pennylane VQE tutorial and H2 hamiltonian: <a href="https://pennylane.ai/qml/demos/tutorial_vqe">link</a></li>
    <li>Qiskit VQE: <a href="https://qiskit-community.github.io/qiskit-algorithms/tutorials/03_vqe_simulation_with_noise.html">link</a></li>
    <li>Algorithms for finding ground and excited states: <a href="https://quantaggle.com/algorithms/algorithm/">link</a></li>   
</ol>

### Versions
<hr>

In [56]:
import sys
print(sys.version)

3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]


In [57]:
import pkg_resources

# Loop through installed packages and filter those starting with 'qiskit'
for dist in pkg_resources.working_set:
    if dist.project_name.lower().startswith("qiskit"):
        print(f"{dist.project_name} == {dist.version}")

qiskit == 1.4.4
qiskit-aer == 0.17.1
qiskit-algorithms == 0.4.0
qiskit-nature == 0.7.2
qiskit-nature-pyscf == 0.4.0
qiskit-qasm3-import == 0.6.0
